## Realizando Imports

In [176]:
import re
import unicodedata
import pandas as pd
from unidecode import unidecode
from unicodedata import normalize

## Leituras dos CSV files

In [177]:
df_escolas = pd.read_csv("escolas.csv")
df_material = pd.read_csv("material_didatico.csv")
df_sub = pd.read_csv("subprefeituras.csv")

## Definições das funções

In [178]:

def short_to_full_tag(original_str: str) -> str:
    address_mapping = {
        "R.": "Rua",
        "Av.": "Avenida",
        "Pça.": "Praça",
        "Estr.": "Estrada"
    }
    
    short_tag, address = original_str.split(maxsplit=1)
    if short_tag in address_mapping.keys():
        return f"{address_mapping[short_tag]} {address}"
    else:
        return original_str
    
def fix_names(name: str) -> str:
    replacements = {
        "E.M.": "ESCOLA MUNICIPAL",
        "CIEP": "CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA",
        "EM": "ESCOLA MUNICIPAL",
        "e.m.": "ESCOLA MUNICIPAL",
        "E.M": "ESCOLA MUNICIPAL"
    }

    for abbreviation, replacement in replacements.items():
        if name.startswith(abbreviation):
            return name.replace(abbreviation, replacement, 1)

    return name

def type_school(name: str) -> str:
    if "ESCOLA MUNICIPAL" in name:
        return "EM"
    elif "CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA" in name:
        return "CIEP"
    elif "COLÉGIO" in name:
        return "COLÉGIO"
    else:
        return "Desconhecido"
    
def remove_acentos(txt: str) -> str:
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')




In [179]:
normalized_address_list = []
address_names = []
address_numbers = []

In [180]:
df_escolas.columns = ["id", "escolas_postos", "bairro", "endereco", "lat", "lon"]
address_list = df_escolas["endereco"].values.tolist()
df_escolas["endereco"] = df_escolas["endereco"].apply(short_to_full_tag).str.upper()

for address in df_escolas["endereco"].values.tolist():
    normalized_address = unicodedata.normalize("NFKD", address).encode("ASCII", errors="ignore").decode("ASCII")
    
    normalized_address_list.append(normalized_address.replace(",", ""))
    
df_escolas["endereco"] = normalized_address_list
df_escolas.head()

for i, address in enumerate(df_escolas["endereco"].values.tolist()):
    address_name = re.sub(r"\d+|S/No", "", address)
    try:
        address_number = re.search(r"\d+|S/No", address).group().replace("S/No", "S/N")
    except AttributeError:
        address_number = "S/N"

    address_names.append(address_name.strip())
    address_numbers.append(address_number.strip())
    
df_escolas["logradouro"] = address_names
df_escolas["numero_end"] = address_numbers

df_escolas['id'] = df_escolas['id'].apply(lambda x: str(x).zfill(3))

df_escolas.loc[df_escolas['bairro']=='RECREIO','bairro'] = 'RECREIO DOS BANDEIRANTES'
df_escolas.loc[df_escolas['bairro']=='RIO DAS PEDRAS', 'bairro'] = 'JACAREPAGUA'
df_escolas.loc[df_escolas['bairro']=='FREGUESIA JPA', 'bairro'] = 'Freguesia (Jacarepaguá)'
df_escolas.loc[df_escolas['bairro']=='AUGUSTO VASCONCELOS', 'bairro'] = 'Santíssimo'
df_escolas.loc[df_escolas['bairro']=='NOVA SEPETIBA', 'bairro'] = 'Sepetiba'
df_escolas.loc[df_escolas['bairro'].str.contains('MARACANÃ'), 'bairro'] = 'Vila Isabel'

df_escolas['bairro'] = df_escolas['bairro'].apply(remove_acentos)
df_escolas['bairro'] = df_escolas['bairro'].str.upper()

df_escolas['escolas_postos'] = df_escolas['escolas_postos'].apply(fix_names)
df_escolas['tipo_escola'] = df_escolas['escolas_postos'].apply(type_school)

df_escolas["bairro"] = df_escolas["bairro"].str.upper().apply(unidecode)

df_escolas.head()

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero_end,tipo_escola
0,178,CENTRO INTEGRADO DE EDUCAÇÃO PÚBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/No,"-22,880888","-43,225326",RUA CARLOS SEIDL,S/N,CIEP
1,634,ESCOLA MUNICIPAL ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202",RUA EBANO,187,EM
2,483,ESCOLA MUNICIPAL CELESTINO SILVA,CENTRO,RUA DO LAVRADIO 56,"-22,909293","-43,183579",RUA DO LAVRADIO,56,EM
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456",PRACA ARGENTINA,20,EM
4,132,ESCOLA MUNICIPAL PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579",PRACA CONDESSA PAULO DE FRONTIN,45,EM


In [181]:
#nome
df_sub.loc[df_sub['nome'] == 'Osvaldo Cruz', 'nome'] = 'OSWALDO CRUZ'
df_sub['nome'] = df_sub['nome'].str.upper()
df_sub['nome'] = df_sub['nome'].apply(remove_acentos)

# subprefeitura
df_sub['subprefeitura'] = df_sub['subprefeitura'].str.upper()


In [183]:

df_material.rename(columns=
    {
        'Id': 'id',
        'Quantidade': 'quantidade',
    }, inplace=True)

df_material['id'] = df_material['id'].apply(lambda x: str(x).zfill(3))

df_material['quantidade'] = df_material['quantidade'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
df_material_invalido = df_material[df_material['quantidade'] == '']
df_material = df_material[df_material['quantidade'] != '']


In [184]:
df_merged = df_escolas.merge(df_sub, left_on='bairro', right_on='nome', how='left')
df_merged.drop(columns=['nome'], inplace=True)  
df_merged = df_merged.merge(df_material, left_on="id", right_on="id", how="left")


In [185]:
rows = df_merged[df_merged['subprefeitura'].isna()]

In [186]:
rows

,id,escolas_postos,bairro,endereco,lat,lon,logradouro,numero_end,tipo_escola,subprefeitura,quantidade


In [187]:
df_merged['quantidade'] = pd.to_numeric(df_merged['quantidade'])
df_material_por_subprefeitura = df_merged.groupby(df_merged['subprefeitura']).agg({'quantidade': 'sum'}).reset_index()
df_material_por_subprefeitura

,subprefeitura,quantidade
0,BARRA DA TIJUCA,266.0
1,CENTRO,3771.0
2,GRANDE BANGU,4115.0
3,GRANDE TIJUCA,2616.0
4,ILHAS,741.0
5,JACAREPAGUÁ,2973.0
6,ZONA NORTE,16665.0
7,ZONA OESTE,9027.0
8,ZONA SUL,4304.0
